# Query Transformation Techniques - llamaindex

A user query can be transformed and decomposed in many ways before being executed as part of a RAG query engine, agent, or any other pipeline.

In this guide we show you different ways to transform, decompose queries, and find the set of relevant tools. Each technique might be applicable for different use cases!

Here are the different query transformations:

1. **Query-Rewriting**: Keep the tools, but rewrite the query in a variety of different ways to execute against the same tools.
2. **Sub-Questions**: Decompose queries into multiple sub-questions over different tools (identified by their metadata).
3. **Multi-Step Query Decomposition** : A multi-step query engine that's able to decompose a complex query into sequential subquestions



In [1]:
%pip install llama-index-question-gen-openai
%pip install llama-index-llms-openai
%pip install llama-index

In [2]:
from IPython.display import Markdown, display


# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

In [3]:
import os
from getpass import getpass

# platform.openai.com
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)


Enter OpenAI API Key: ··········


## Query Rewriting

In this section, we show you how to rewrite queries into multiple queries. You can then execute all these queries against a retriever.

This is a key step in advanced retrieval techniques. By doing query rewriting, you can generate multiple queries for [ensemble retrieval] and [fusion], leading to higher-quality retrieved results.

Unlike the sub-question generator, this is just a prompt call, and exists independently of tools.

### Query Rewriting (Custom)

Here we show you how to use a prompt to generate multiple queries, using our LLM and prompt abstractions.

In [4]:
from llama_index.core import PromptTemplate
from llama_index.llms.openai import OpenAI

query_gen_str = """\
You are a helpful assistant that generates multiple search queries based on a \
single input query. Generate {num_queries} search queries, one on each line, \
related to the following input query:
Query: {query}
Queries:
"""
query_gen_prompt = PromptTemplate(query_gen_str)

llm = OpenAI(model="gpt-3.5-turbo")


def generate_queries(query: str, llm, num_queries: int = 4):
    response = llm.predict(
        query_gen_prompt, num_queries=num_queries, query=query
    )
    # assume LLM proper put each query on a newline
    queries = response.split("\n")
    queries_str = "\n".join(queries)
    print(f"Generated queries:\n{queries_str}")
    return queries

In [5]:
queries = generate_queries("What happened at Citrix", llm)

Generated queries:
1. Latest news on Citrix incident
2. Citrix security breach details
3. Impact of Citrix data breach
4. Citrix response to recent events


### Query Rewriting (using HyDE Query Transform)

In this section we show you how to do query transformations using our Hyde QueryTransform class.

In [6]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from IPython.display import Markdown, display

In [7]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2024-02-27 11:01:31--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.01s   

2024-02-27 11:01:31 (5.64 MB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



In [8]:
# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

In [9]:
index = VectorStoreIndex.from_documents(documents)


First, we query without transformation: The same query string is used for embedding lookup and also summarization.

In [10]:
query_str = "what did paul graham do after going to RISD"

In [11]:
query_engine = index.as_query_engine()
response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

<b>Paul Graham started painting still lives in his bedroom at night while he was a student at the Accademia di Belli Arti in Florence.</b>

Now, we use HyDEQueryTransform to generate a hypothetical document and use it for embedding lookup.

In [12]:
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

<b>After going to RISD, Paul Graham resumed his old patterns, experimented with a new kind of still life painting technique, looked for an apartment to buy, and eventually had the idea to build a web app for making web apps. He got excited about this idea and decided to move to Cambridge to start a new company focused on this concept.</b>

In [13]:
query_bundle = hyde(query_str)
hyde_doc = query_bundle.embedding_strs[0]

In [14]:
hyde_doc

"After attending the Rhode Island School of Design (RISD), Paul Graham went on to co-found Viaweb, one of the first online stores that allowed users to build their own e-commerce websites. Viaweb was later acquired by Yahoo for $49 million, marking Graham's first successful venture in the tech industry. Following this success, Graham went on to co-found Y Combinator, a startup accelerator that has helped launch numerous successful companies such as Dropbox, Airbnb, and Reddit. Graham is also known for his influential essays on startups and entrepreneurship, as well as his work as an angel investor in various tech companies. Overall, Paul Graham's career after RISD has been marked by a series of successful ventures and contributions to the tech industry."

## Sub-Questions

Given a set of tools and a user query, decide both the 1) set of sub-questions to generate, and 2) the tools that each sub-question should run over.

We run through an example using the `OpenAIQuestionGenerator`, which depends on function calling, and also the `LLMQuestionGenerator`, which depends on prompting.

In [15]:
from llama_index.core.question_gen import LLMQuestionGenerator
from llama_index.question_gen.openai import OpenAIQuestionGenerator
from llama_index.llms.openai import OpenAI

In [16]:
llm = OpenAI()
question_gen = OpenAIQuestionGenerator.from_defaults(llm=llm)

In [17]:
display_prompt_dict(question_gen.get_prompts())

**Prompt Key**: question_gen_prompt<br>**Text:** <br>

You are a world class state of the art agent.

You have access to multiple tools, each representing a different data source or API.
Each of the tools has a name and a description, formatted as a JSON dictionary.
The keys of the dictionary are the names of the tools and the values are the descriptions.
Your purpose is to help answer a complex user question by generating a list of sub questions that can be answered by the tools.

These are the guidelines you consider when completing your task:
* Be as specific as possible
* The sub questions should be relevant to the user question
* The sub questions should be answerable by the tools provided
* You can generate multiple sub questions for each tool
* Tools must be specified by their name, not their description
* You don't need to use a tool if you don't think it's relevant

Output the list of sub questions by calling the SubQuestionList function.

## Tools
```json
{tools_str}
```

## User Question
{query_str}



<br><br>

In [18]:
from llama_index.core.tools import ToolMetadata

tool_choices = [
    ToolMetadata(
        name="uber_2021_10k",
        description=(
            "Provides information about Uber financials for year 2021"
        ),
    ),
    ToolMetadata(
        name="lyft_2021_10k",
        description=(
            "Provides information about Lyft financials for year 2021"
        ),
    ),
]

In [19]:
from llama_index.core import QueryBundle

query_str = "Compare and contrast Uber and Lyft"
choices = question_gen.generate(tool_choices, QueryBundle(query_str=query_str))

The outputs are `SubQuestion` Pydantic objects.

In [20]:
choices

[SubQuestion(sub_question='What were the total revenues for Uber in 2021?', tool_name='uber_2021_10k'),
 SubQuestion(sub_question='What were the total revenues for Lyft in 2021?', tool_name='lyft_2021_10k'),
 SubQuestion(sub_question='What were the net profits for Uber in 2021?', tool_name='uber_2021_10k'),
 SubQuestion(sub_question='What were the net profits for Lyft in 2021?', tool_name='lyft_2021_10k')]

# Multi-Step Query Decomposition

We have a multi-step query engine that's able to decompose a complex query into sequential subquestions. This
guide walks you through how to set it up!

#### Download Data

In [21]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2024-02-27 11:01:47--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.01s   

2024-02-27 11:01:47 (5.51 MB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



#### Load documents, build the VectorStoreIndex

In [22]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display

In [23]:
# LLM (gpt-3.5)
gpt35 = OpenAI(temperature=0, model="gpt-3.5-turbo")



In [24]:
# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

In [25]:
index = VectorStoreIndex.from_documents(documents)

#### Query Index

In [26]:
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)

step_decompose_transform = StepDecomposeQueryTransform(llm=gpt35, verbose=True)

# gpt-3
step_decompose_transform_gpt3 = StepDecomposeQueryTransform(
    llm=gpt35, verbose=True
)

In [27]:
index_summary = "Used to answer questions about the author"

In [28]:
# set Logging to DEBUG for more detailed outputs
from llama_index.core.query_engine import MultiStepQueryEngine

query_engine = index.as_query_engine(llm=gpt35)
query_engine = MultiStepQueryEngine(
    query_engine=query_engine,
    query_transform=step_decompose_transform,
    index_summary=index_summary,
)
response_gpt_1 = query_engine.query(
    "Who was in the first batch of the accelerator program the author"
    " started?",
)

> Current query: Who was in the first batch of the accelerator program the author started?
> New query: Who started the accelerator program?
> Current query: Who was in the first batch of the accelerator program the author started?
> New query: Who was in the first batch of the accelerator program started by Paul Graham?
> Current query: Who was in the first batch of the accelerator program the author started?
> New query: Who were some of the individuals in the first batch of the accelerator program started by Paul Graham?


In [29]:
display(Markdown(f"<b>{response_gpt_1}</b>"))

<b>The first batch of the accelerator program the author started included reddit, Justin Kan and Emmett Shear, Aaron Swartz, and Sam Altman.</b>

In [30]:
sub_qa = response_gpt_1.metadata["sub_qa"]
tuples = [(t[0], t[1].response) for t in sub_qa]
print(tuples)

[('Who started the accelerator program?', 'Paul Graham started the accelerator program.'), ('Who was in the first batch of the accelerator program started by Paul Graham?', 'The first batch of the accelerator program started by Paul Graham included reddit, Justin Kan and Emmett Shear, Aaron Swartz, and Sam Altman.'), ('Who were some of the individuals in the first batch of the accelerator program started by Paul Graham?', 'The first batch of the accelerator program started by Paul Graham included reddit, Justin Kan and Emmett Shear, Aaron Swartz, and Sam Altman.')]


In [31]:
response_gpt_1 = query_engine.query(
    "In which city did the author found his first company, Viaweb?",
)

> Current query: In which city did the author found his first company, Viaweb?
> New query: Where did the author found his first company, Viaweb?
> Current query: In which city did the author found his first company, Viaweb?
> New query: Where was the author working out of when he founded his first company, Viaweb?
> Current query: In which city did the author found his first company, Viaweb?
> New query: Where was Robert's apartment located when the author founded his first company, Viaweb?


In [32]:
print(response_gpt_1)

Cambridge


In [33]:
query_engine = index.as_query_engine(llm=gpt35)
query_engine = MultiStepQueryEngine(
    query_engine=query_engine,
    query_transform=step_decompose_transform_gpt3,
    index_summary=index_summary,
)

response_gpt3 = query_engine.query(
    "In which city did the author found his first company, Viaweb?",
)

> Current query: In which city did the author found his first company, Viaweb?
> New query: Where did the author found his first company, Viaweb?
> Current query: In which city did the author found his first company, Viaweb?
> New query: Where was the author working when he founded his first company, Viaweb?
> Current query: In which city did the author found his first company, Viaweb?
> New query: Where was the author's friend Robert's apartment located when he founded his first company, Viaweb?


In [34]:
print(response_gpt3)

The author founded his first company, Viaweb, in Cambridge.
